In [ ]:
from query.datasets.prelude import *
from query.datasets.ingest import ingest_pose

In [ ]:
# Video.objects.annotate(
#                 c=Subquery(
#                     Shot.objects.filter(video=OuterRef('pk')).values('video') \
#                     .annotate(c=Count('video')).values('c')
#                 )).filter(c__isnull=False).values('path') 
# videos = Video.objects.filter(height__gt=480)
# for v in videos:
#     v.height = 480
#     v.save()

In [ ]:
video_map = {v.path: v for v in Video.objects.all()}
with open('/app/all_videos_dl.txt') as f:
    paths = ['tvnews/videos/{}.mp4'.format(s.strip()) for s in f.readlines()]
videos = [video_map[path] for path in paths]

In [ ]:
with Database(prefetch_table_metadata=False) as db:
    tables = [db.table(v.path + '_shots') 
             for v in videos if db.has_table(v.path + '_shots') and db.table(v.path + '_shots').committed()]
    print(len(tables))
    
    def load(t):
        try:
            return pickle.loads(next(t.column('shots').load(rows=[t.num_rows() - 1]))[1])
        except Exception:
            traceback.print_exc()
            print(t.name())
            return None
    
    shots = par_for(load, tables)
    print('Done!')

In [ ]:
pickle.dump([(t.name(), l) for t, l in zip(tables, shots)], open('all_shots.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)

In [ ]:
as1 = pickle.load(open('all_shots.pkl', 'rb'))
as2 = pcache.get('all_shots_2')

In [ ]:
def bulk_load(db, videos):
    shots = [
        db.table(v.path + '_shots') for v in videos
        if db.has_table(v.path + '_shots') and db.table(v.path + '_shots').committed()
    ]

    def load(t):
        try:
            return pickle.loads(next(t.column('shots').load(rows=[t.num_rows() - 1]))[1])
        except Exception:
            traceback.print_exc()
            print(t.name())
            return None

    return par_for(load, shots)

log.debug('Loading videos')
videos = list(Video.objects.annotate(
                c=Subquery(
                    Shot.objects.filter(video=OuterRef('pk')).values('video') \
                    .annotate(c=Count('video')).values('c')
                )).filter(c__isnull=False))
log.debug('Connecting to db')
with Database() as db:
    log.debug('Loading data')
    db._load_db_metadata()
    shots = bulk_load(db, videos)

In [ ]:
Frame.objects.filter(tags__name='openpose:labeled').count()

In [ ]:
pose = Pose.objects.get(id=1625727)
pose.pose_keypoints()[Pose.Nose]

In [ ]:
from query.datasets.tvnews.face_embed import face_embed

video = Video.objects.get(path='tvnews/videos/MSNBC_20100827_060000_The_Rachel_Maddow_Show.mp4')
face = Face.objects.get(id=119091)
FaceFeatures.objects.filter(face=face).delete()
face_embed([video], [[[face]]], force=True)


In [ ]:
face = Face.objects.get(id=119083)
FaceFeatures.objects.get(face=face).id

In [ ]:
db = Database()

In [ ]:
#Person.objects.filter(frame__video__path='tvnews/videos/MSNBC_20090703_010000_The_Rachel_Maddow_Show.mp4').delete()
#Face.objects.all().count()
#Video.objects.all().distinct('height').values('height')

with open('/app/paths') as f:
    paths = [s.strip() for s in f.readlines()]
    
tag, _ = Tag.objects.get_or_create(name='pose-test')    
for path in paths:
    video = Video.objects.get(path=path)
    VideoTag(video=video, tag=tag).save()

In [ ]:
Video.objects.filter(videotag__tag__name='pose-test').values('path')

In [ ]:
#FaceFeatures.objects.filter(face__person__frame__video__videotag__tag__name='pose-test').count()
#print Face.objects.all().order_by('facefeatures__distto').values('id').query
#PersonTrack.objects.filter(video__path='tvnews/videos/MSNBC_20100827_060000_The_Rachel_Maddow_Show.mp4') \
#    .annotate(duration=Track.duration()).values()

#id = 2043
#FaceFeatures.compute_distances(id)
tracks = list(PersonTrack.objects.filter(video__videotag__tag__name='pose-test') \
#tracks = list(PersonTrack.objects.filter(video__path='tvnews/videos/MSNBCW_20120303_020000_The_Rachel_Maddow_Show.mp4') \
    .annotate(c=Subquery(
        Face.objects.filter(person__tracks=OuterRef('pk')) \
        .filter(labeler__name='tinyfaces', facefeatures__distto__isnull=False, facefeatures__distto__lte=1.0) \
        .values('person__tracks')
        .annotate(c=Count('*'))
        .values('c'), models.IntegerField()
        )) \
    .filter(c__gt=0))
len(tracks)

In [ ]:
frames = defaultdict(set)    
for track in tracks:
    frames[track.video.path] |= set(range(track.min_frame, track.max_frame, 3))

# for path in frames.keys():    
#     if path == 'tvnews/videos/MSNBCW_20120303_020000_The_Rachel_Maddow_Show.mp4': continue
#     print(path)
#     #print('Deleting')
#     #Person.objects.filter(frame__video=Video.objects.get(path=path))
#     #Pose.objects.filter(person__frame__video=Video.objects.get(path=path)).delete()
#     print('Ingesting')
#     ingest_pose(
#         Video.objects.get(path=path),
#         db.table(path + '_poses_gather'),
#         sorted(frames[path]))
#     print('Ingested!')

with open('/app/chris-matthews-frames', 'w') as f:
    for path, frames in frames.iteritems():
        print(path)
        f.write('{} {}\n'.format(path, ' '.join([str(s) for s in sorted(frames)])))

In [ ]:
def closest_pose(candidates, target):
    noses = [pose.pose_keypoints()[Pose.Nose] for pose in candidates]
    noses, indices = zip(*[(nose[:2], i) for i, nose in enumerate(noses) if nose[2] > 0])
    dists = np.linalg.norm(np.array(noses) - target, axis=1)
    closest = candidates[indices[np.argmin(dists)]]
    return closest

In [ ]:
# Match detected poses with the sparse faces 
for track in tracks:
    faces = list(Face.objects.filter(person__tracks=track).select_related('person__frame'))
    for face in faces:
        poses = list(Pose.objects.filter(person__frame=face.person.frame))
        if len(poses) == 0:
            continue
        closest = closest_pose(poses, bbox_midpoint(face))
        closest.person = face.person
        closest.save()

In [ ]:
# Fill in tracks with poses
for track in tracks:
    faces = list(Face.objects.filter(person__tracks=track).select_related('person__frame') \
                 .order_by('person__frame__number'))
    for i in range(len(faces) - 1):
        try:
            last_pose = Pose.objects.get(person=faces[i].person)
            for j in range(faces[i].person.frame.number, faces[i+1].person.frame.number, 3):
                cur_poses = Pose.objects.filter(person__frame__video=track.video, person__frame__number=j)
                if len(cur_poses) == 0:
                    continue
                closest = closest_pose(cur_poses, last_pose.pose_keypoints()[Pose.Nose][:2])
                closest.person.tracks.add(track)
                closest.save()
                last_pose = closest
        except Pose.DoesNotExist:
            pass      

In [ ]:
for track in tracks:
    print(track.id, track.max_frame-track.min_frame, Person.objects.filter(tracks=track).count())

In [ ]:
# TODO: need to associate faces with poses
[p1, p2] = Pose.objects.all()[:2]

def pose_dist(p1, p2):
    kp1 = p1.pose_keypoints()
    kp2 = p2.pose_keypoints()
    
    weights = defaultdict(float, {
        Pose.LWrist: 0.4,
        Pose.RWrist: 0.4,
        Pose.Nose: 0.1,        
        Pose.LElbow: 0.05,
        Pose.RElbow: 0.05
    })
    weight_vector = [weights[i] for i in range(Pose.POSE_KEYPOINTS)]
    
    dist = np.linalg.norm(kp2[:,:2] - kp1[:,:2], axis=1)
    weighted_dist = np.array([d * w for d, s, w in zip(dist, kp2[:, 2], weight_vector) if s > 0])
    return np.linalg.norm(weighted_dist)

In [ ]:
all_dists = []
for track in tracks:
    poses = list(Pose.objects.filter(person__tracks=track).order_by('person__frame__number'))
    dists = [pose_dist(poses[i], poses[i+1]) for i in range(len(poses) - 1)]
    all_dists.append((track.id, np.mean(dists), track.duration()))

pprint(sorted(all_dists, key=itemgetter(1), reverse=True))